In [ ]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_tags, strip_punctuation, strip_numeric, stem_text
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_non_alphanum, remove_stopwords, strip_short
from gensim import corpora, models, similarities
from gensim.utils import simple_preprocess
from smart_open import smart_open
import csv
from gensim.models import CoherenceModel
import pandas as pd
import os, json
import time
import ast

import errno
import glob
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim
import nltk

nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer
lemmatizer = WordNetLemmatizer()
snowball = SnowballStemmer('english')


     |████████████████████████████████| 1.6MB 2.7MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97712 sha256=8dc47e0ea46cc6414ab35aaac7941b5a0c49644a9c7a8a69daa83a293301beae
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
Successfully built pyLDAvis
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
custom_filters = [lambda filename: filename.lower(), #lowercase
                  strip_multiple_whitespaces, #remove repeating whitespaces
                  strip_numeric, #remove numbers
                  remove_stopwords, #remove stopwords
                  strip_short, #remove words less than 3 characters long
                  strip_punctuation #remove all punctuations 
                  ]

In [ ]:
def lem_stem(word):
  word = lemmatizer.lemmatize(word)
  word = snowball.stem(word)
  return word 

In [ ]:
def gensim_process(json_file):

  # Declaring global variable that can be used outside functions
  global corpus
  global dictionary

  dataframe = pd.read_json(json_file, lines = True)
  dataframe = dataframe.to_string()

  # preprocess the text through the custom filter 
  gensim_processed = preprocess_string(dataframe, custom_filters) 

  # lemmatize and then stem the words
  processed = ' '.join([lem_stem(word) for word in gensim_processed])

  #filter again after lemma+stem  
  final_processed = preprocess_string(processed, custom_filters)

  # split the string into a list to enable the tokens to be converted into a dictionary 
  dataset = [d.split() for d in final_processed]


  # convert the data into a dictionary to be able to process the data through gensim
  dictionary = corpora.Dictionary(dataset)

  # Convert document (a list of words) into the bag-of-words format
  corpus = [dictionary.doc2bow(text) for text in dataset]



In [ ]:
def visualize(corpus, dictionary, main, year):
  start = time.time()

  #initialize the tfidf model 
  tfidf = models.TfidfModel(corpus)

  #apply the transofmration to the entire corpus 
  transformed_tfidtf = tfidf[corpus]

  #use the LDA transformation on tfidf
  lda = models.LdaMulticore(transformed_tfidtf, num_topics=8, id2word=dictionary)
  lda.show_topics()

  #visualizing the topics
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(lda, corpus, dictionary)
  print(time.time()- start)

  #saving data
  with open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/{}/vis_file_{}_{}.html'.format(main,year,main,year), 'w') as file: 
    pyLDAvis.save_html(vis, file)

  csvfile = open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/{}/corpus_{}_{}.csv'.format(main,year,main,year), 'w')
  csvwriter = csv.writer(csvfile)
  for item in corpus:
    csvwriter.writerow(item)
  csvfile.close()

  w = csv.writer(open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/{}/dictionary_{}_{}.csv'.format(main,year,main,year), "w"))
  for key, val in dictionary.items():
    w.writerow([key, val])

In [ ]:
# Asks the user for the name of the main folder and changes to its directory
#main_folder = input('Enter the main folder to run: ')
main_folder = input('Enter main folder to run: ')
folder_path = "/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}".format(main_folder)


years = [folder for folder in os.listdir(folder_path)]
print (years)



corpus_external = []
dictionary_external = corpora.Dictionary()
start = time.time()


for year in years:
  year_path = "/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/{}".format(main_folder, year)
  
  json_files = [pos_json for pos_json in
              os.listdir(year_path) if pos_json.endswith('.json')]

  for json_file in json_files:
    file_path = year_path + '/{}'.format(json_file)
    print (file_path)
    gensim_process(file_path)
    corpus_external.extend(corpus)
    
    dictionary_external.merge_with(dictionary)



    
    

    
   

print('\n Corpus and Dictionary created\n')
print(dictionary_external)
print(time.time()- start)


Enter main folder to run: ANJO
['1818', '1817', '1819', '1820', '1821', '1822', '1823', '1824', '1825', '1858', '1859', '1860']
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1818/02_11.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1817/07_30.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1819/03_17.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1819/06_30.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1820/07_26.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1820/11_29.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1820/06_07.json
/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/ANJO/1820/08_16.json
/content/drive/My Drive/

KeyboardInterrupt: ignored

In [ ]:

start = time.time()

#initialize the tfidf model 
tfidf = models.TfidfModel(corpus_external)

#apply the transofmration to the entire corpus 
transformed_tfidtf = tfidf[corpus_external]

#use the LDA transformation on tfidf
lda = models.LdaMulticore(transformed_tfidtf, num_topics=8, id2word=dictionary_external)
lda.show_topics()

#visualizing the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda, corpus_external, dictionary_external)

print(time.time()- start)

In [ ]:
#Run this for visualization
vis

In [ ]:
#extract the visualization data 
with open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/vis_file_{}.html'.format(main_folder, main_folder), 'w') as file: 
    pyLDAvis.save_html(vis, file)

   

In [ ]:

csvfile = open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/corpus_{}.csv'.format(main_folder, main_folder), 'w')
csvwriter = csv.writer(csvfile)
for item in corpus_external:
    csvwriter.writerow(item)
csvfile.close()


In [ ]:
w = csv.writer(open('/content/drive/My Drive/Cochrane Master Folder/Specific Articles Related to Cochrane/{}/dictionary_{}.csv'.format(main_folder, main_folder), "w"))
for key, val in dictionary_external.items():
  w.writerow([key, val])


